In [0]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import asyncio
import chess
import chess.engine

spark = (
    SparkSession.builder
    .appName('bq')
    .config('parentProject', 'chessthesis')
    .config('chessthesis-account-databricks.json', 'dbfs:FileStore/tables/bigquery_service_account/chessthesis_account_databricks.json') \
    .getOrCreate()
)



In [0]:
file_location = '/dbfs/FileStore/tables/bigquery_service_account/chessthesis_account_databricks.json'

df = spark.read.format("bigquery").option("credentialsFile", file_location).option("project", "chessthesis").option("table","chessthesis.chess_thesis.test_evaluations").load()

In [0]:
async def analyse(position):
    transport, engine = await chess.engine.popen_uci("/dbfs/FileStore/tables/stockfish_engine/stockfish_14_1_linux_x64_popcnt")
    board = chess.Board(position)
    info = await engine.analyse(board, chess.engine.Limit(depth=20))
    
    await engine.quit()
    
    return info["score"].pov("WHITE").score()

def analyse_position(position, engine):
    board = chess.Board(position)
    info = engine.analyse(board, chess.engine.Limit(depth=20))
    return info["score"].pov("WHITE").score()

engine = chess.engine.SimpleEngine.popen_uci("/dbfs/FileStore/tables/stockfish_engine/stockfish_14_1_linux_x64_popcnt")
print(analyse_position("r1b1kb1r/pp2pppp/1qn2n2/3p4/2pP1B2/2P1P3/PPQN1PPP/R3KBNR b KQkq - 1 7", engine))

/databricks/python/lib/python3.8/site-packages/chess/engine.py:154: RuntimeWarning: A loop is being detached from a child watcher with pending handlers
 warnings.warn("A loop is being detached from a child watcher with pending handlers", RuntimeWarning)

--------------------------------------------------------------------------- 
 TimeoutError Traceback (most recent call last)
 <command-287288043780449> in <module> 
 13 return info [ "score" ] . pov ( "WHITE" ) . score ( ) 
 14 
 ---> 15 engine = chess . engine . SimpleEngine . popen_uci ( "/dbfs/FileStore/tables/stockfish_engine/stockfish_14_1_linux_x64_popcnt" ) 
 16 print ( analyse_position ( "r1b1kb1r/pp2pppp/1qn2n2/3p4/2pP1B2/2P1P3/PPQN1PPP/R3KBNR b KQkq - 1 7" , engine ) ) 

 /databricks/python/lib/python3.8/site-packages/chess/engine.py in popen_uci (cls, command, timeout, debug, setpgrp, **popen_args) 
 2823 Returns a : class : ` ~ chess . engine . SimpleEngine ` instance . 
 2824 """
 -> 2825 return cls . popen ( UciProtocol , command , timeout = timeout , debug = debug , setpgrp = setpgrp , ** popen_args ) 
 2826 
 2827 @ classmethod

 /databricks/python/lib/python3.8/site-packages/chess/engine.py in popen (cls, Protocol, command, timeout, debug, setpgrp, **popen_args) 
 2815 await simple_engine . shutdown_event . wait ( ) 
 2816 
 -> 2817 return run_in_background ( background , name = f"{cls.__name__} (command={command!r})" , debug = debug ) 
 2818 
 2819 @ classmethod

 /databricks/python/lib/python3.8/site-packages/chess/engine.py in run_in_background (coroutine, name, debug, _policy_lock) 
 199 
 200 threading . Thread ( target = background , name = name ) . start ( ) 
 --> 201 return future . result ( ) 
 202 
 203 

 /usr/lib/python3.8/concurrent/futures/_base.py in result (self, timeout) 
 442 raise CancelledError ( ) 
 443 elif self . _state == FINISHED : 
 --> 444 return self . __get_result ( ) 
 445 else : 
 446 raise TimeoutError ( ) 

 /usr/lib/python3.8/concurrent/futures/_base.py in __get_result (self) 
 387 if self . _exception : 
 388 try : 
 --> 389 raise self . _exception
 390 finally : 
 391 # Break a reference cycle with the exception in self._exception 

 /databricks/python/lib/python3.8/site-packages/chess/engine.py in background () 
 193 def background ( ) -> None : 
 194 try : 
 --> 195 asyncio . run ( coroutine ( future ) ) 
 196 future . cancel ( ) 
 197 except Exception as exc : 

 /usr/lib/python3.8/asyncio/runners.py in run (main, debug) 
 42 if debug is not None : 
 43 loop . set_debug ( debug ) 
 ---> 44 return loop . run_until_complete ( main ) 
 45 finally : 
 46 try : 

 /usr/lib/python3.8/asyncio/base_events.py in run_until_complete (self, future) 
 614 raise RuntimeError ( 'Event loop stopped before Future completed.' ) 
 615 
 --> 616 return future . result ( ) 
 617 
 618 def stop ( self ) : 

 /databricks/python/lib/python3.8/site-packages/chess/engine.py in background (future) 
 2807 simple_engine = cls ( transport , protocol , timeout = timeout ) 
 2808 try : 
 -> 2809 await asyncio . wait_for ( protocol . initialize ( ) , timeout ) 
 2810 future . set_result ( simple_engine ) 
 2811 returncode = await protocol . returncode

 /usr/lib/python3.8/asyncio/tasks.py in wait_for (fut, timeout, loop) 
 499 # See https://bugs.python.org/issue32751 
 500 await _cancel_and_wait ( fut , loop = loop ) 
 --> 501 raise exceptions . TimeoutError ( ) 
 502 finally : 
 503 timeout_handle . cancel ( ) 

 TimeoutError :

In [0]:
analyse_position("r1b1kb1r/pp2pppp/1qn2n2/3p4/2pP1B2/2P1P3/PPQN1PPP/R3KBNR b KQkq - 1 7")

Out[13]: 31

In [0]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql.functions import *

def analyse_position(position):
    print(position)
    engine = chess.engine.SimpleEngine.popen_uci("dbfs:/FileStore/tables/stockfish_engine/stockfish_14_1_linux_x64_popcnt'")
    board = chess.Board(position)
    info = engine.analyse(board, chess.engine.Limit(depth=18))
    try:
        return info["score"].pov("WHITE").score()
    except:
        return -999
    
def test_fun(string):
    return 1

udf_analyse = udf(lambda x: analyse_position(x), IntegerType())
df1 = df.limit(1)
#display(df1)
df1 = df1.withColumn("eval", udf_analyse(col("string_field_2")))
display(df1)


org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 4) (ip-10-83-227-125.eu-central-1.compute.internal executor driver): org.apache.spark.api.python.PythonException: 'FileNotFoundError: [Errno 2] No such file or directory: "dbfs:/FileStore/tables/stockfish_engine/stockfish_14_1_linux_x64_popcnt'"', from , line 7. Full traceback below:
Traceback (most recent call last):
 File " ", line 18, in 
 File " ", line 7, in analyse_position
 File "/databricks/python/lib/python3.8/site-packages/chess/engine.py", line 2825, in popen_uci
 return cls.popen(UciProtocol, command, timeout=timeout, debug=debug, setpgrp=setpgrp, **popen_args)
 File "/databricks/python/lib/python3.8/site-packages/chess/engine.py", line 2817, in popen
 return run_in_background(background, name=f"{cls.__name__} (command={command!r})", debug=debug)
 File "/databricks/python/lib/python3.8/site-packages/chess/engine.py", line 201, in run_in_background
 return future.result()
 File "/usr/lib/python3.8/concurrent/futures/_base.py", line 449, in result
 self = None
 File "/usr/lib/python3.8/concurrent/futures/_base.py", line 392, in __get_result
 self = None
 File "/databricks/python/lib/python3.8/site-packages/chess/engine.py", line 198, in background
 future.set_exception(exc)
 File "/usr/lib/python3.8/asyncio/runners.py", line 51, in run
 loop.close()
 File "/usr/lib/python3.8/asyncio/base_events.py", line 616, in run_until_complete
 return future.result()
 File "/databricks/python/lib/python3.8/site-packages/chess/engine.py", line 2805, in background
 transport, protocol = await Protocol.popen(command, setpgrp=setpgrp, **popen_args)
 File "/databricks/python/lib/python3.8/site-packages/chess/engine.py", line 1214, in popen
 return await asyncio.get_running_loop().subprocess_exec(cls, *command, **popen_args) # type: ignore
 File "/usr/lib/python3.8/asyncio/base_events.py", line 1630, in subprocess_exec
 transport = await self._make_subprocess_transport(
 File "/usr/lib/python3.8/asyncio/unix_events.py", line 211, in _make_subprocess_transport
 raise
 File "/usr/lib/python3.8/asyncio/base_subprocess.py", line 40, in __init__
 raise
 File "/usr/lib/python3.8/asyncio/unix_events.py", line 799, in _start
 stdin_w.close()
 File "/usr/lib/python3.8/subprocess.py", line 892, in __init__
 raise
 File "/usr/lib/python3.8/subprocess.py", line 1704, in _execute_child
 raise child_exception_type(errno_num, err_msg, err_filename)
FileNotFoundError: [Errno 2] No such file or directory: "dbfs:/FileStore/tables/stockfish_engine/stockfish_14_1_linux_x64_popcnt'"

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:641)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:86)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:68)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:594)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:489)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:757)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:80)
	at org.apache.spark.sql.execution.collect.Collector.$anonfun$processFunc$1(Collector.scala:178)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$3(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfile

In [0]:


positions = [item[0] for item in df.select("string_field_2").distinct().collect()][0:20]
par_positions = sc.parallelize(positions)
#print(par_positions.collect())
my_list = par_positions.collect()
print(my_list)

#evals = []
evals = par_positions.map(lambda x: analyse(x))
print(evals)
#evals.collect()
#evals.saveAsSequenceFile("/sparktest.txt")
#sorted(sc.sequenceFile("/sparktest.txt/").take(10))

['rbbq1rk1/pp3ppp/2p5/3nn3/3N4/1P2P1N1/PBQ1BPPP/R4RK1 w - - 3 15', '3r1b1r/p1k2ppp/bpp1p3/2n1P3/5B2/5B2/PPPN1PPP/R3R1K1 w - - 6 18', '6k1/p4pp1/2B4p/1pb2P2/6b1/2P3B1/r5PP/4R2K b - - 0 31', 'rnbqkb1r/pp2pppp/3p1n2/8/3NP3/8/PPP2PPP/RNBQKB1R w KQkq - 1 5', 'r1bqr1k1/pp1n1pbp/2pp2p1/5p2/2PP4/2N1P1P1/PP2NPBP/R2Q1RK1 w - - 2 11', '3rr1k1/1p1nbpp1/2p1p1p1/p6q/1PPP4/P4NP1/1BR1QPP1/3R2K1 w - - 0 20', 'r1q2rk1/pb3pp1/1n5p/2ppP2Q/1n6/2N3PN/PP3PBP/R4RK1 b - - 1 18', '7k/6p1/7p/1P1p4/3Qb3/7P/6PK/1r6 w - - 2 46', '4k2r/1Q1qb2p/p2p2pB/2p1p3/r1P1P2P/8/P5P1/3R1RK1 w k - 4 25', 'r4rk1/pb1q1p2/4pp1p/2pP4/8/P5P1/1P2PPBP/R2Q1RK1 b - - 0 15', '1rbq1rk1/p1p1bppp/2p1pn2/8/Q1pP4/4P1P1/PP3PBP/RNB2RK1 b - - 2 10', 'rnbqkb1r/pp2pppp/5n2/2pp4/3P1B2/4PN2/PPP2PPP/RN1QKB1R b KQkq - 1 4', 'r3k2r/ppp1qppp/1nn1bb2/1B2p3/1PP5/P2P1N2/3N1PPP/R1BQR1K1 w kq - 1 13', 'r2qbrk1/1p2b3/3p1n1p/3Pp3/pPN1Ppp1/P1NQ4/2R2PPP/1R2B1K1 w - - 2 23', '3rr3/2n2bbk/pp2p2p/2B3p1/P1P1Np2/3P1P2/1PB1R1PP/4R2K w - - 0 34', '3r1n2/1p4p1/1p3pk1/1Bb1p2p/1P2P2P/2P2PB1/P3K1P1/7R b - - 0 25', 'rnbqkb1r/pp2pppp/2p2n2/3p4/2P5/4PN2/PP1P1PPP/RNBQKB1R w KQkq - 1 4', '2r4k/1q1np2p/1p1p1p2/p2P4/P2BP1p1/1P3P2/3Q4/6RK w - - 2 34', 'rnbqkbnr/pp2pppp/2p5/3p4/3PP3/8/PPPN1PPP/R1BQKBNR b KQkq - 1 3', 'r7/p2rnpk1/1p1p2pp/2P1p3/2P1P3/P1NPP1P1/6KP/1R1R4 b - - 0 24']
PythonRDD[48] at RDD at PythonRDD.scala:58

In [0]:
display(df)

fide_id,name,federation,gender,title,yob
100145,"Felice, Carolina",ARG,F,null,0
112020,"Herrera, Carla",ARG,F,null,0
113727,"Romano, Julieta",ARG,F,null,0
114448,"Lopez Nocera, Ariadna",ARG,F,null,0
115827,"Tournier, Virginia",ARG,F,null,0
116971,"Sanseverino, Victoria",ARG,F,null,0
118923,"Diez Lopez, Veronica",ARG,F,null,0
124800,"Besozzi, Maria",ARG,F,null,0
126390,"Minervino, Paula",ARG,F,null,0
126667,"Fraire, Gisela",ARG,F,null,0


In [0]:
%sh
unzip /FileStore/tables/stockfish/stockfish_14_1_linux_x64_popcnt.zip

unzip: cannot find or open /FileStore/tables/stockfish/stockfish_14_1_linux_x64_popcnt.zip, /FileStore/tables/stockfish/stockfish_14_1_linux_x64_popcnt.zip.zip or /FileStore/tables/stockfish/stockfish_14_1_linux_x64_popcnt.zip.ZIP.

Score: PovScore(Mate(+1), WHITE)